# KSA: Polar Observations and Modelling

# Week 6 - Introduction to an ocean/sea ice model

Now that we have done a little overview of ACCESS-OM2 and some of its data, it's time to do a bit of exploring into this model output. 

For this, you can find in the `data/` directory, output for the Southern Ocean that includes temperature and salinity fields, sea surface height, and surface currents. All this variables come in monthly averages, from 1980 to 2018. We have also included a bit of information on the ocean's grid and bathymetry in the `grid.nc` file. 

Use the notebooks and code from the last four classes, as well as [CoPilot](https://copilot.microsoft.com/) to select a region of the Southern Ocean and investigate some aspect of the ocean state/circulation. You are welcome to choose your own path, but you can also choose from any of the suggested "mini" research projects.

The data available for you to use includes:
 - Model's bathymetry -> `data/access-om2_depth.nc`
 - 3D oxygen fields, averaged during the period 2000-2018 -> `data/access-om2_o2_avg_2000-2018.nc`
 - Sea ice concentration seasonal cycle for the period 2000-2018 -> `data/access-om2_sea_ice_cc_clim_2000-2018.nc`
 - Sea level averaged for the period 2000-2018 -> `data/access-om2_sea_level_avg_2000-2018.nc`
 - 3D temperature and salinity fields, averaged during the period 2000-2018 -> `data/access-om2_temp_salt_avg_2000-2018.nc`
 - 3D temperature and salinity fields and mixed layer depth seasonal cycle for the period 2000-2018 -> `data/access-om2_temp_salt_mld_clim_2000-2018.nc`
 - 3D zonal and meridional velocities, averaged during the period 2000-2018 -> `data/access-om2_u_v_avg_2000-2018.nc`

In [1]:
import cartopy.crs as ccrs
import cartopy.feature as cft
import cmocean as cm
import glob
import gsw
import matplotlib.lines as mlines
import matplotlib.path as mpath
import matplotlib.pyplot as plt
import metpy
import numpy as np
import xarray as xr
import xarrayMannKendall as xrmk

In [2]:
def stereographic_axis(ax):
    
    ax.set_extent([-180, 180, -80, -35], crs = ccrs.PlateCarree())
    ax.add_feature(cft.NaturalEarthFeature('physical', 'land', '50m',
                                            edgecolor='gainsboro',
                                            facecolor='gainsboro'))
    ax.coastlines(resolution = "110m")
    # Map the plot boundaries to a circle
    theta = np.linspace(0, 2 * np.pi, 100)
    center, radius = [0.5, 0.5], 0.5
    verts = np.vstack([np.sin(theta), np.cos(theta)]).T
    circle = mpath.Path(verts * radius + center)
    ax.set_boundary(circle, transform=ax.transAxes)

    return ax

## Comparison of temperature and salinity between West and East Antarctica

<div style="float: right; margin-right: 20px; margin-left: 30px; width: 20%; max-width: 500px;">
    <img src="images/ant_melt.png" width="100%" />
</div>

The West Antarctic Ice Sheet (WAIS) has been loosing mass in a way the East Antarctic has not (take a look at the figure from [Smith et al. 2020](https://www.science.org/doi/10.1126/science.aaz5845)). The higher vulnerability of WAIS to melt has been linked to warmer ocean temperatures at the continental shelf related to intrusions of Circumpolar Deep Water from the open ocean.

Use the temperature and salinity fields from the ACCESS-OM2 model (the file `data/access_om2_temp_salt_avg_2000-2018.nc` contains this variables averaged over the 2000-2018 period) to compare the temperature and salinity of the continental shelf at West Antarctica vs the one at East Antarctica. The variable `temp` is conservative temperature and comes in units of Kelvin, and the variable `salt` is practical salinity and comes in units of PSU. You could:
 - Do TS diagrams
 - Plot maps of temperature/salinity at different depths
 - Etc

In order to do this, you will have to be able to select the continental shelf in a region, which is slightly more nuanced than just selecting a latitude/longitude box. Take inspiration from the second class, where we looked the Antarctic Bottom Water layer selecting those densities larger than 28.27 kg/m$^3$: if the continental shelf is defined as the area shallower than 1000m, you can use the `xarray.where()` function to select the continental shelf. For example:

```
xr.where(ht<1000, temp, np.nan)
```

where we are saying that where the depth `ht` is less than a 1000, we want to keep the temperature values, otherwise we want `NaN`s. 

Before doing the above, you will want to have pre-selected a `temp_WA` variable for the West Antarctic region, and a `temp_EA` for the East Antarctic region. Look at the figure from Smith et al. 2020 to do the selection, and remember that the bigger the area you choose, the longer your calculation might take!

## Identifying Southern Ocean fronts from sea level

<div style="float: right; margin-right: 20px; margin-left: 30px; width: 20%; max-width: 500px;">
    <img src="images/acc_fronts_schem.png" width="100%" />
</div>

An oceanographic front is a sharp boundary that separates two areas with water masses of different characteristics. Fronts are a really important feature of the ocean circulation: they sustain strong currents, they give rise to instabilities and eddies and they are usually biologically productive regions, rich in nutrients (see the figure on the left from [Chapman et al. (2020)](https://www.nature.com/articles/s41558-020-0705-4). These fronts also exhert an obstacle (a barrier) for tracers to move from one side to the other. 

Within the Southern Ocean there are several of these fronts. Three of the most important ones are the Subantarctic Front, the Polar Front and the Southern ACC Front. Traditionally, these have been identified from sea surface height data, using a specific value of an SSH contour associated to each front. However, for some applications, like looking at changes in frontal position throughout time, instead of choosing a fixed value, another definition may be used based on the *gradient* of sea level. Essentially identifying a front as a region of *large change in small distances* rather than a fixed value. Remember that, like we saw in `2_SO_circulation_from_geostrophy.ipynb`, regions with large gradients are associated to strong currents!

Use the `sea_level` variable from the model, which you will find in `data/access_om2_sea_level_avg_2000-2018.nc` and look at the position of the three fronts using the following contours: -0.51 for the Subantarctic Front (SAF); -0.96 for the Polar Front (PF) and -1.5 for the Southern ACC Front (SB). Use temperature, salinity (in `data/access_om2_temp_salt_avg_2000-2018.nc`) and oxygen (in `data/access_om2_o2_avg_2000-2018.nc` to look at the subsurface beneath those front definitions. 

You could plot a cross section of temp/salt/oxygen and mark the location of the fronts. Or you could plot a map with contours for the front locations, and either of these variables at a depth of your choice.

## Identify three different shelf and Antarctic Slope Current regimes

The Antarctic Slope Current (ASC) is a quasi-circumpolar current that flows around Antarctica in an anti-clockwise direction. We say it is quasi-circumpolar because it is either absent or very weak in West Antarctica. It is a really important dynamical feature of the ocean circulation, because typically when there is a strong, established current, the exchange of water masses from either side becomes inhibited - in other words, the ASC acts as a barrier for open-ocean water masses to come onto the shelf. Remember that open-ocean water masses are usually warmer than the waters on shelf, and so this mediating role controls to a certain extant how much heat gets to the ice shelves. The lack (or weakness) of the ASC in West Antarctica is one of the reasons behind the vulnerability of this area and the increased melt rates observed here. 

The ASC's properties vary around the Antarctic margins, and it is usually classified into three distinct regimes: a surface intensified, a bottom intensified, and a reverse ASC. These are illustrated in the figure below from [Huneke et al. (2022)](https://journals.ametsoc.org/view/journals/phoc/52/3/JPO-D-21-0143.1.xml):

<p align="center">
<img src="images/asc_regimes.png" width="70%"/>
</p>

Each of these regimes is associated with a particular temperature/salinity structure of the Antarctic Slope Front - fresh ASF, dense ASF and warm ASF. You can see the spatial distribution of these three regimes in the following figure from [Thompson et al. (2018)](https://agupubs.onlinelibrary.wiley.com/doi/full/10.1029/2018RG000624):

<p align="center">
<img src="images/asf_regimes.png" width="40%"/>
</p>

Use the velocity, temperature and salinity fields to select and plot three cross-sections that are representative of the three ASC/ASF regimes. These variables are in `data/access-om2_temp_salt_avg_2000-2018.nc` and `data/access-om2_u_v_avg_2000-2018.nc`, averaged for the 2000-2018 period. Calculate and plot potential density too!

In order to do this, you will have to use `xarray.sel()` to select the area of interested. You can try with a single location, or averaging over a slice. Think about which dimension you will be making your selection along!

## Seasonality of the surface layer and open ocean polynyas

We have seen in previous classes how the formation of bottom waters (termed Antarctic Bottom Water) occurs in localised regions around the Antarctic margins. The complexity of the processes involved means that it is a process that few numerical models are able to reproduce accurately: most models produce bottom waters by open-ocean convection, large factories of bottom waters off-shelf in the Weddell and Ross Seas. Open-ocean convection is a phenomena that has rarely been observed: in fact, we have only observed it twice, in the event of the [Maud Rise polynya](https://earthobservatory.nasa.gov/images/145069/deciphering-the-maud-rise-polynya) that opened up in the 1970s and in 2016.

One of the unique and valuable features of ACCESS-OM2 at its highest, 0.1$^{\circ}$ resolution, is that it does not have these polynyas and forms bottom waters on shelf, in the observed locations and in right amounts. The two lower resolutions, 0.25$^{\circ}$ and 1$^{\circ}$ (the latter is the one we will use here), do have spurious open ocean convection.

Take a look at the temperature, salinity, mixed layer depth and sea ice concentration seasonal cycle. See if you can identify these open ocean polynyas by looking at the mixed layer depth - remembering that the convection in these polynyas will appear as an abnormally large mixed layer depth. How does the sesasonal cycle in areas of with open ocean convection, and areas with no open ocean convection, differ?

You can try plotting mixed layer depth for each month to identify areas with convection. Once you've identified them, calculate temperature, salinity and sea ice concentration averages. How are these related? Since you have 3D temperature and salinity fields, you can either try doing a Hovmoller plot (like in `1.Introduction_to_EN422.ipynb`, or just pick a depth of your choice and plot a time series. 

The variables you will need are in `data/access-om2_temp_salt_mld_clim_2000-2018.nc` (where you have `temp`, `sa;t` and `mld` or mixed layer depth) and `data/access-om2_sea_ice_cc_clim_2000-2018.nc` (where you have `aice_m` or sea ice concentration).